In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv('/content/Flipkart Review.csv')
train.head()

,review,rating
0,It was nice produt. I like it's design a lot. ...,0
1,awesome sound....very pretty to see this nd th...,0
2,awesome sound quality. pros 7-8 hrs of battery...,0
3,I think it is such a good product not only as ...,0
4,awesome bass sound quality very good bettary l...,0


In [4]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [5]:
train['Tidy_Reviews'] = np.vectorize(remove_pattern)(train['review'], "@[\w]*")
train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")
train['Tidy_Reviews'] = train['Tidy_Reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

<ipython-input-5-be17ecef7fa3>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")


,review,rating,Tidy_Reviews
0,It was nice produt. I like it's design a lot. ...,0,nice produt like design easy carry Looked styl...
1,awesome sound....very pretty to see this nd th...,0,awesome sound very pretty this sound quality g...
2,awesome sound quality. pros 7-8 hrs of battery...,0,awesome sound quality pros battery life includ...
3,I think it is such a good product not only as ...,0,think such good product only quality also desi...
4,awesome bass sound quality very good bettary l...,0,awesome bass sound quality very good bettary l...


In [6]:
tokenized_review = train['Tidy_Reviews'].apply(lambda x: x.split())
tokenized_review.head()

0    [nice, produt, like, design, easy, carry, Look...
1    [awesome, sound, very, pretty, this, sound, qu...
2    [awesome, sound, quality, pros, battery, life,...
3    [think, such, good, product, only, quality, al...
4    [awesome, bass, sound, quality, very, good, be...
Name: Tidy_Reviews, dtype: object

In [7]:
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_review = tokenized_review.apply(lambda x: [ps.stem(i) for i in x])
tokenized_review.head()

0    [nice, produt, like, design, easi, carri, look...
1    [awesom, sound, veri, pretti, thi, sound, qual...
2    [awesom, sound, qualiti, pro, batteri, life, i...
3    [think, such, good, product, onli, qualiti, al...
4    [awesom, bass, sound, qualiti, veri, good, bet...
Name: Tidy_Reviews, dtype: object

In [8]:
for i in range(len(tokenized_review)):
    tokenized_review[i] = ' '.join(tokenized_review[i])

train['Tidy_Reviews'] = tokenized_review
train.head()

,review,rating,Tidy_Reviews
0,It was nice produt. I like it's design a lot. ...,0,nice produt like design easi carri look stylis...
1,awesome sound....very pretty to see this nd th...,0,awesom sound veri pretti thi sound qualiti goo...
2,awesome sound quality. pros 7-8 hrs of battery...,0,awesom sound qualiti pro batteri life includ m...
3,I think it is such a good product not only as ...,0,think such good product onli qualiti also desi...
4,awesome bass sound quality very good bettary l...,0,awesom bass sound qualiti veri good bettari lo...


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['Tidy_Reviews'])

df_bow = pd.DataFrame(bow.todense())

df_bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
tfidf=tfidf.fit_transform(train['Tidy_Reviews'])
df_tfidf = pd.DataFrame(tfidf.todense())
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.146096,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
train_bow = bow[:9976]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [12]:
train_tfidf= tfidf[:9976]

train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,train['rating'],test_size=0.3,random_state=2)
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf,train['rating'],test_size=0.3,random_state=17)

In [15]:
pip install keras.preprocessing

In [16]:
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [17]:
x_train_bow = x_train_bow.toarray()
x_valid_bow = x_valid_bow.toarray()

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_bow.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))  # Additional Convolutional Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))  # Additional Dense Layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bow, y_train_bow, batch_size=5, epochs=15)


Epoch 1/15
1397/1397 [==============================] - 84s 59ms/step - loss: 0.2453 - accuracy: 0.9157
Epoch 2/15
1397/1397 [==============================] - 44s 32ms/step - loss: 0.1938 - accuracy: 0.9311
Epoch 3/15
1397/1397 [==============================] - 41s 30ms/step - loss: 0.1705 - accuracy: 0.9348
Epoch 4/15
1397/1397 [==============================] - 44s 31ms/step - loss: 0.1480 - accuracy: 0.9449
Epoch 5/15
1397/1397 [==============================] - 54s 39ms/step - loss: 0.1259 - accuracy: 0.9519
Epoch 6/15
1397/1397 [==============================] - 42s 30ms/step - loss: 0.1038 - accuracy: 0.9588
Epoch 7/15
1397/1397 [==============================] - 42s 30ms/step - loss: 0.0909 - accuracy: 0.9663
Epoch 8/15
1397/1397 [==============================] - 44s 32ms/step - loss: 0.0776 - accuracy: 0.9699
Epoch 9/15
1397/1397 [==============================] - 42s 30ms/step - loss: 0.0682 - accuracy: 0.9774
Epoch 10/15
1397/1397 [==============================] - 43s 30m

In [19]:
score = model.evaluate(x_valid_bow, y_valid_bow, verbose=1)

94/94 [==============================] - 3s 25ms/step - loss: 0.3681 - accuracy: 0.9158


In [20]:
x_train_tfidf = x_train_tfidf.toarray()
x_valid_tfidf = x_valid_tfidf.toarray()

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_tfidf.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))  # Additional Convolutional Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))  # Additional Dense Layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_tfidf, y_train_tfidf, batch_size=5, epochs=15)


Epoch 1/15
1397/1397 [==============================] - 43s 30ms/step - loss: 0.2415 - accuracy: 0.9131
Epoch 2/15
1397/1397 [==============================] - 44s 31ms/step - loss: 0.1929 - accuracy: 0.9277
Epoch 3/15
1397/1397 [==============================] - 43s 31ms/step - loss: 0.1702 - accuracy: 0.9360
Epoch 4/15
1397/1397 [==============================] - 41s 30ms/step - loss: 0.1483 - accuracy: 0.9472
Epoch 5/15
1397/1397 [==============================] - 48s 35ms/step - loss: 0.1241 - accuracy: 0.9553
Epoch 6/15
1397/1397 [==============================] - 44s 32ms/step - loss: 0.1054 - accuracy: 0.9629
Epoch 7/15
1397/1397 [==============================] - 41s 30ms/step - loss: 0.0867 - accuracy: 0.9692
Epoch 8/15
1397/1397 [==============================] - 41s 30ms/step - loss: 0.0739 - accuracy: 0.9739
Epoch 9/15
 555/1397 [==========>...................] - ETA: 26s - loss: 0.0598 - accuracy: 0.9791

KeyboardInterrupt: ignored

In [ ]:
score = model.evaluate(x_valid_tfidf, y_valid_tfidf, verbose=1)